In [2]:
%%writefile practice9.cpp

#include <mpi.h> // подключает MPI
#include <iostream> // подключает вывод
#include <vector> // подключает vector
#include <cmath> // подключает sqrt и fabs
#include <cstdlib> // подключает atoi и rand
#include <iomanip> // подключает setprecision
#include <algorithm> // подключает min

using namespace std; // чтобы не писать std::

int ceil_div(int a, int b) { // деление с округлением вверх
    return (a + b - 1) / b; // формула
}

void fill_random_double(vector<double>& a, int n, int rank) { // заполняет массив случайными double
    if (rank != 0) return; // делает только rank 0
    srand(123); // фиксирует seed
    for (int i = 0; i < n; i++) { // цикл по массиву
        a[i] = (double)(rand() % 1000) / 10.0; // числа 0..99.9
    }
}

void fill_system(vector<double>& A, vector<double>& b, int n, int rank) { // создает систему Ax=b
    if (rank != 0) return; // делает только rank 0
    srand(123); // фиксирует seed
    for (int i = 0; i < n; i++) { // строки
        for (int j = 0; j < n; j++) { // столбцы
            double x = (double)(rand() % 10 + 1); // 1..10
            if (i == j) x += n; // усиливает диагональ чтобы было устойчивее
            A[i * n + j] = x; // кладет в матрицу
        }
        b[i] = (double)(rand() % 10 + 1); // правая часть
    }
}

void fill_graph(vector<int>& G, int n, int rank) { // заполняет матрицу смежности для графа
    if (rank != 0) return; // делает только rank 0
    srand(123); // фиксирует seed
    const int INF = 1000000000; // большое число как бесконечность
    for (int i = 0; i < n; i++) { // строки
        for (int j = 0; j < n; j++) { // столбцы
            if (i == j) { // диагональ
                G[i * n + j] = 0; // расстояние до себя = 0
            } else { // иначе
                int r = rand() % 10; // 0..9
                if (r < 3) G[i * n + j] = INF; // иногда ребра нет
                else G[i * n + j] = 1 + rand() % 20; // вес 1..20
            }
        }
    }
}

   // TASK 1: MEAN + STDDEV
void task1_mean_std(int N, int rank, int size) { // задача 1
    vector<double> full; // полный массив только у rank 0
    if (rank == 0) full.assign(N, 0.0); // выделяет память
    fill_random_double(full, N, rank); // заполняет только rank 0

    vector<int> counts(size, 0); // сколько элементов каждому процессу
    vector<int> displs(size, 0); // смещения в массиве
    int base = N / size; // базовая часть
    int rem = N % size; // остаток

    for (int r = 0; r < size; r++) { // цикл по процессам
        counts[r] = base + (r < rem ? 1 : 0); // раздает остаток первым rem процессам
    }

    displs[0] = 0; // смещение для rank 0
    for (int r = 1; r < size; r++) { // считает смещения
        displs[r] = displs[r - 1] + counts[r - 1]; // сумма предыдущих
    }

    int local_n = counts[rank]; // сколько элементов у текущего процесса
    vector<double> local(local_n, 0.0); // локальный кусок

    double t1 = MPI_Wtime(); // старт времени

    MPI_Scatterv( // раздает массив с учетом остатка
        rank == 0 ? full.data() : nullptr, // sendbuf только у root
        counts.data(), // sendcounts
        displs.data(), // displs
        MPI_DOUBLE, // тип данных
        local.data(), // recvbuf
        local_n, // recvcount
        MPI_DOUBLE, // тип данных
        0, // root
        MPI_COMM_WORLD // коммуникатор
    );

    double local_sum = 0.0; // сумма локальных элементов
    double local_sumsq = 0.0; // сумма квадратов локальных элементов

    for (int i = 0; i < local_n; i++) { // цикл по локальному массиву
        double x = local[i]; // берет элемент
        local_sum += x; // добавляет в сумму
        local_sumsq += x * x; // добавляет квадрат
    }

    double global_sum = 0.0; // общая сумма
    double global_sumsq = 0.0; // общая сумма квадратов

    MPI_Reduce(&local_sum, &global_sum, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD); // собирает суммы
    MPI_Reduce(&local_sumsq, &global_sumsq, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD); // собирает суммы квадратов

    double t2 = MPI_Wtime(); // конец времени

    if (rank == 0) { // печатает только root
        double mean = global_sum / (double)N; // среднее
        double var = (global_sumsq / (double)N) - mean * mean; // дисперсия
        if (var < 0) var = 0; // защита от отрицательного из-за double
        double stddev = sqrt(var); // стандартное отклонение

        cout << "TASK 1: mean + stddev" << endl; // заголовок
        cout << "N = " << N << endl; // размер
        cout << "processes = " << size << endl; // число процессов
        cout << fixed << setprecision(6); // формат
        cout << "mean = " << mean << endl; // вывод среднего
        cout << "stddev = " << stddev << endl; // вывод std
        cout << "mpi time = " << (t2 - t1) << " sec" << endl; // время
        cout << endl; // пустая строка
    }
}

  // TASK 2: GAUSS (distributed forward, root backward)
void task2_gauss(int N, int rank, int size) { // задача 2
    int rows_per = ceil_div(N, size); // сколько строк на процесс (с округлением вверх)
    int Npad = rows_per * size; // дополняем матрицу до кратности size
    int local_rows = rows_per; // локальные строки
    int local_A_elems = local_rows * N; // сколько элементов матрицы у процесса

    vector<double> A_full; // матрица A только у root
    vector<double> b_full; // вектор b только у root
    vector<double> A_pad; // паддинг матрицы только у root
    vector<double> b_pad; // паддинг вектора только у root

    if (rank == 0) { // только root
        A_full.assign(N * N, 0.0); // выделяет A
        b_full.assign(N, 0.0); // выделяет b
        fill_system(A_full, b_full, N, rank); // заполняет систему

        A_pad.assign(Npad * N, 0.0); // выделяет A с паддингом
        b_pad.assign(Npad, 0.0); // выделяет b с паддингом

        for (int i = 0; i < N; i++) { // копирует реальные строки
            for (int j = 0; j < N; j++) { // копирует столбцы
                A_pad[i * N + j] = A_full[i * N + j]; // копия A
            }
            b_pad[i] = b_full[i]; // копия b
        }
    }

    vector<double> A_local(local_A_elems, 0.0); // локальная часть A
    vector<double> b_local(local_rows, 0.0); // локальная часть b

    double t1 = MPI_Wtime(); // старт времени

    MPI_Scatter( // раздает строки матрицы A
        rank == 0 ? A_pad.data() : nullptr, // sendbuf
        local_A_elems, // sendcount
        MPI_DOUBLE, // тип
        A_local.data(), // recvbuf
        local_A_elems, // recvcount
        MPI_DOUBLE, // тип
        0, // root
        MPI_COMM_WORLD // comm
    );

    MPI_Scatter( // раздает куски вектора b
        rank == 0 ? b_pad.data() : nullptr, // sendbuf
        local_rows, // sendcount
        MPI_DOUBLE, // тип
        b_local.data(), // recvbuf
        local_rows, // recvcount
        MPI_DOUBLE, // тип
        0, // root
        MPI_COMM_WORLD // comm
    );

    vector<double> pivot(N + 1, 0.0); // pivot строка + элемент b

    for (int k = 0; k < N; k++) { // шаг прямого хода
        int owner = k / rows_per; // какой процесс владеет строкой k
        int owner_i = k % rows_per; // локальный индекс строки у владельца

        if (rank == owner) { // владелец готовит pivot
            for (int j = 0; j < N; j++) { // копирует строку
                pivot[j] = A_local[owner_i * N + j]; // берет элемент
            }
            pivot[N] = b_local[owner_i]; // кладет b
        }

        MPI_Bcast(pivot.data(), N + 1, MPI_DOUBLE, owner, MPI_COMM_WORLD); // рассылает pivot всем

        double piv = pivot[k]; // ведущий элемент
        if (fabs(piv) < 1e-12) { // если pivot почти ноль
            if (rank == 0) cout << "warning: near-zero pivot at k=" << k << endl; // предупреждение
            continue; // пропускает шаг (упрощенно)
        }

        for (int i = 0; i < local_rows; i++) { // цикл по локальным строкам
            int global_i = rank * rows_per + i; // глобальный индекс строки
            if (global_i <= k) continue; // только строки ниже pivot
            if (global_i >= N) continue; // пропускает паддинг строки

            double factor = A_local[i * N + k] / piv; // коэффициент вычитания
            A_local[i * N + k] = 0.0; // зануляет элемент под диагональю

            for (int j = k + 1; j < N; j++) { // обновляет хвост строки
                A_local[i * N + j] -= factor * pivot[j]; // вычитает
            }
            b_local[i] -= factor * pivot[N]; // обновляет b
        }
    }

    vector<double> A_gather; // собранная матрица только у root
    vector<double> b_gather; // собранный b только у root

    if (rank == 0) { // только root
        A_gather.assign(Npad * N, 0.0); // выделяет память
        b_gather.assign(Npad, 0.0); // выделяет память
    }

    MPI_Gather( // собирает A
        A_local.data(), // sendbuf
        local_A_elems, // sendcount
        MPI_DOUBLE, // тип
        rank == 0 ? A_gather.data() : nullptr, // recvbuf
        local_A_elems, // recvcount
        MPI_DOUBLE, // тип
        0, // root
        MPI_COMM_WORLD // comm
    );

    MPI_Gather( // собирает b
        b_local.data(), // sendbuf
        local_rows, // sendcount
        MPI_DOUBLE, // тип
        rank == 0 ? b_gather.data() : nullptr, // recvbuf
        local_rows, // recvcount
        MPI_DOUBLE, // тип
        0, // root
        MPI_COMM_WORLD // comm
    );

    double t2 = MPI_Wtime(); // конец времени

    if (rank == 0) { // обратный ход делает root
        vector<double> x(N, 0.0); // решение

        for (int i = N - 1; i >= 0; i--) { // идем снизу вверх
            double sum = b_gather[i]; // правая часть
            for (int j = i + 1; j < N; j++) { // вычитает известные x
                sum -= A_gather[i * N + j] * x[j]; // вычитание
            }
            double diag = A_gather[i * N + i]; // диагональ
            if (fabs(diag) < 1e-12) diag = 1e-12; // защита
            x[i] = sum / diag; // вычисляет x[i]
        }

        cout << "TASK 2: Gauss (simple)" << endl; // заголовок
        cout << "N = " << N << endl; // размер
        cout << "processes = " << size << endl; // процессы
        cout << "mpi time = " << (t2 - t1) << " sec" << endl; // время
        cout << fixed << setprecision(6); // формат
        cout << "first 10 x:" << endl; // заголовок
        for (int i = 0; i < min(N, 10); i++) { // печатает первые 10
            cout << "x[" << i << "] = " << x[i] << endl; // вывод
        }
        cout << endl; // пустая строка
    }
}


 //  TASK 3: FLOYD-WARSHALL (simple Allgather each step)

void task3_floyd(int N, int rank, int size) { // задача 3
    const int INF = 1000000000; // бесконечность
    int rows_per = ceil_div(N, size); // строк на процесс
    int Npad = rows_per * size; // паддинг строк
    int local_rows = rows_per; // локальные строки

    vector<int> G_full; // полный граф только у root
    vector<int> G_pad; // паддинг граф только у root

    if (rank == 0) { // root
        G_full.assign(N * N, 0); // выделяет граф
        fill_graph(G_full, N, rank); // заполняет граф
        G_pad.assign(Npad * N, INF); // выделяет паддинг

        for (int i = 0; i < Npad; i++) { // строки паддинга
            for (int j = 0; j < N; j++) { // столбцы
                if (i < N) G_pad[i * N + j] = G_full[i * N + j]; // реальные строки
                else G_pad[i * N + j] = (i == j ? 0 : INF); // лишние строки
            }
        }
    }

    vector<int> local(local_rows * N, INF); // локальные строки матрицы
    vector<int> all_mat(Npad * N, INF); // полная матрица у всех процессов

    double t1 = MPI_Wtime(); // старт времени

    MPI_Scatter( // раздает строки
        rank == 0 ? G_pad.data() : nullptr, // sendbuf
        local_rows * N, // sendcount
        MPI_INT, // тип
        local.data(), // recvbuf
        local_rows * N, // recvcount
        MPI_INT, // тип
        0, // root
        MPI_COMM_WORLD // comm
    );

    MPI_Allgather( // собирает матрицу у всех
        local.data(), // sendbuf
        local_rows * N, // sendcount
        MPI_INT, // тип
        all_mat.data(), // recvbuf
        local_rows * N, // recvcount
        MPI_INT, // тип
        MPI_COMM_WORLD // comm
    );

    for (int k = 0; k < N; k++) { // основной цикл Флойда
        for (int i = 0; i < local_rows; i++) { // локальные строки
            int gi = rank * rows_per + i; // глобальная строка
            if (gi >= N) continue; // пропускает паддинг строки

            int ik = all_mat[gi * N + k]; // dist(i,k)
            if (ik >= INF) continue; // если нет пути

            for (int j = 0; j < N; j++) { // столбцы
                int kj = all_mat[k * N + j]; // dist(k,j)
                if (kj >= INF) continue; // если нет пути
                int nd = ik + kj; // новый путь
                int& cur = local[i * N + j]; // ссылка на текущий
                if (nd < cur) cur = nd; // обновляет если лучше
            }
        }

        MPI_Allgather( // обменивается обновленными строками
            local.data(), // sendbuf
            local_rows * N, // sendcount
            MPI_INT, // тип
            all_mat.data(), // recvbuf
            local_rows * N, // recvcount
            MPI_INT, // тип
            MPI_COMM_WORLD // comm
        );
    }

    vector<int> G_out; // итоговая матрица только у root
    if (rank == 0) G_out.assign(Npad * N, INF); // выделяет память

    MPI_Gather( // собирает на root
        local.data(), // sendbuf
        local_rows * N, // sendcount
        MPI_INT, // тип
        rank == 0 ? G_out.data() : nullptr, // recvbuf
        local_rows * N, // recvcount
        MPI_INT, // тип
        0, // root
        MPI_COMM_WORLD // comm
    );

    double t2 = MPI_Wtime(); // конец времени

    if (rank == 0) { // печатает только root
        cout << "TASK 3: Floyd-Warshall (simple)" << endl; // заголовок
        cout << "N = " << N << endl; // размер
        cout << "processes = " << size << endl; // процессы
        cout << "mpi time = " << (t2 - t1) << " sec" << endl; // время
        cout << "top-left 8x8:" << endl; // вывод кусочка
        int lim = min(N, 8); // ограничение
        for (int i = 0; i < lim; i++) { // строки
            for (int j = 0; j < lim; j++) { // столбцы
                int v = G_out[i * N + j]; // значение
                if (v >= INF) cout << "inf "; // если бесконечность
                else cout << v << " "; // иначе число
            }
            cout << endl; // новая строка
        }
        cout << endl; // пустая строка
    }
}

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv); // инициализирует MPI

    int rank = 0; // номер процесса
    int size = 1; // количество процессов
    MPI_Comm_rank(MPI_COMM_WORLD, &rank); // получает rank
    MPI_Comm_size(MPI_COMM_WORLD, &size); // получает size

    int task = 1; // какая задача (1/2/3)
    int N = 1000000; // размер по умолчанию

    if (argc >= 2) task = atoi(argv[1]); // читает task из аргумента
    if (argc >= 3) N = atoi(argv[2]); // читает N из аргумента

    if (rank == 0) { // печатает только root
        cout << "Practice 9 (MPI)" << endl; // заголовок
        cout << "task = " << task << endl; // номер задачи
        cout << "N = " << N << endl; // размер
        cout << "np = " << size << endl; // процессов
        cout << endl; // пустая строка
    }

    if (task == 1) task1_mean_std(N, rank, size); // запускает задачу 1
    else if (task == 2) task2_gauss(N, rank, size); // запускает задачу 2
    else if (task == 3) task3_floyd(N, rank, size); // запускает задачу 3
    else { // если неверно
        if (rank == 0) cout << "wrong task: use 1 or 2 or 3" << endl; // сообщение
    }

    MPI_Finalize(); // завершает MPI
    return 0; // завершает программу
}


Overwriting practice9.cpp


In [3]:
%%bash
mpic++ -O2 practice9.cpp -o practice9


In [4]:
%%bash
mpirun --allow-run-as-root --oversubscribe -np 4 ./practice9 1 1000000


Practice 9 (MPI)
task = 1
N = 1000000
np = 4

TASK 1: mean + stddev
N = 1000000
processes = 4
mean = 49.855229
stddev = 28.863924
mpi time = 0.008632 sec



In [5]:
%%bash
mpirun --allow-run-as-root --oversubscribe -np 4 ./practice9 2 400


Practice 9 (MPI)
task = 2
N = 400
np = 4

TASK 2: Gauss (simple)
N = 400
processes = 4
mpi time = 0.0392951 sec
first 10 x:
x[0] = 0.005719
x[1] = -0.001376
x[2] = 0.014748
x[3] = -0.004077
x[4] = 0.005721
x[5] = 0.000698
x[6] = 0.001006
x[7] = -0.005373
x[8] = 0.002946
x[9] = -0.001505



In [6]:
%%bash
mpirun --allow-run-as-root --oversubscribe -np 4 ./practice9 3 300


Practice 9 (MPI)
task = 3
N = 300
np = 4

TASK 3: Floyd-Warshall (simple)
N = 300
processes = 4
mpi time = 0.0779704 sec
top-left 8x8:
0 3 2 2 3 2 2 3 
3 0 3 3 2 3 3 3 
2 2 0 3 3 2 2 2 
2 2 3 0 2 3 2 3 
3 3 3 2 0 2 3 3 
3 3 3 3 1 0 3 2 
2 2 3 2 3 2 0 2 
3 3 3 3 3 3 3 0 

